# andre marroquin
# rodrigo mansilla
# sergio orellana 
# carlos valladares

# Proyecto 2 — Data Science
### link repo: https://github.com/mar22266/PY2-DS/tree/Resultados-Parciales-y-Visualizaciones-Est%C3%A1ticas

branch: Resultados Parciales y Visualizaciones Estáticas

## Fase 2: Investigación de modelos, selección, entrenamiento, evaluación y discusión

**Reto:** CGIAR – Ojos en el Terreno (detección de daños en cultivos)

**Objetivo de esta fase.** Partiendo del EDA, investigar algoritmos, seleccionar candidatos, construir varios modelos que predigan `EXTENT` (severidad % por fila (ID, DAMAGE)), evaluarlos con métricas apropiadas para **regresión**, comparar resultados, justificar la selección final y generar visualizaciones estáticas.

**Tarea de modelado.** Cada fila representa un par `(ID, DAMAGE)` con metadatos (`season`, `growth_stage`, `filename`) y el objetivo `EXTENT` (0–100 en train; ausente en test). En test se debe predecir `EXTENT` por fila, y luego formatear el archivo de envío siguiendo `SampleSubmission.csv`.


## Configuración, imports y utilidades

In [ ]:
# importaciones de librerias
from __future__ import annotations

import os
from pathlib import Path
import re
import json
import warnings
from dataclasses import dataclass
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

# Rutas
DATA_DIR = Path(".")
TRAIN_CSV = DATA_DIR / "Train.csv"
TEST_CSV = DATA_DIR / "Test.csv"
SAMPLE_CSV = DATA_DIR / "SampleSubmission.csv"

OUTPUT_DIR = Path("./outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

plt.rcParams["figure.dpi"] = 120
plt.rcParams["savefig.bbox"] = "tight"

print("RUTAS OK:", TRAIN_CSV.exists(), TEST_CSV.exists(), SAMPLE_CSV.exists())


RUTAS OK: True True True


## Selección de algoritmos a probar (justificación)

Como la variable objetivo `EXTENT` es **continua (0–100)** por `(ID, DAMAGE)`, se plantean **modelos de regresión tabular** con metadatos y rasgos derivados del `filename`:

1) **Ridge Regression** (lineal regularizada): buen baseline, interpretable; sirve para detectar relaciones lineales tras one-hot y scaling.  
2) **Random Forest Regressor**: no lineal, robusto a outlier*, captura interacciones y no exige scaling.  
3) **HistGradientBoostingRegressor** (sklearn): *boosting* eficiente, suele mejorar MAE/RMSE en tabular con variables categóricas codificadas.

Todos evaluados con **GroupKFold por `ID`** para evitar *leakage* entre filas del mismo ID (pues cada imagen aporta varias filas, una por `DAMAGE`).



## Carga y verificación de datos

In [ ]:
train = pd.read_csv(TRAIN_CSV)
test = pd.read_csv(TEST_CSV)
sample = pd.read_csv(SAMPLE_CSV)

print(train.head(3))
print(test.head(3))
print(sample.head(3))
print("shapes:", train.shape, test.shape, sample.shape)


              ID                        filename growth_stage damage  extent  \
0  ID_1S8OOWQYCB  L427F01330C01S03961Rp02052.jpg            S     WD       0   
1  ID_0MD959MIZ0      L1083F00930C39S12674Ip.jpg            V      G       0   
2  ID_JRJCI4Q11V      24_initial_1_1463_1463.JPG            V      G       0   

   season  
0  SR2020  
1  SR2021  
2  LR2020  
              ID                         filename growth_stage damage  season
0  ID_ROOWKB90UZ   L122F00315C01S02151Rp04021.jpg            V     WD  SR2020
1  ID_PTEDRY0CYM  L1089F03254C01S08845Rp25119.jpg            F     WD  LR2021
2  ID_5WJXDV96R4   L365F01913C39S12578Rp42918.jpg            V     WD  SR2021
              ID  extent
0  ID_KJ12GE2U80       0
1  ID_W33POE3DBX       0
2  ID_1DZ7VKQTS9       0
shapes: (26068, 6) (8663, 5) (8663, 2)


## Preprocesamiento columnas, mapeos, features

In [ ]:
# Normaliza códigos de temporada
def normalize_season(s: str) -> str:
    if pd.isna(s): 
        return "UNKNOWN"
    t = str(s).upper().strip().replace(" ", "")
    t = re.sub(r"[^A-Z0-9]", "", t)
    return t

_STAGE_MAP = {
    "F": "F", "FLOWERING": "F",
    "M": "M", "MATURITY": "M",
    "S": "S", "SOWING": "S",
    "V": "V", "VEGETATIVE": "V"
}

# Normaliza códigos de etapa de crecimiento
def normalize_stage(s: str) -> str:
    if pd.isna(s): 
        return "UNK"
    t = str(s).upper().strip()
    return _STAGE_MAP.get(t, t if t in {"F","M","S","V"} else "UNK")

def filename_features(fname: str) -> Dict[str, str|int]:
    """
    Heurística ligera para extraer rasgos del 'filename':
    - prefijos con L###F####C##S#####...
    - números grandes como proxy de sitio/cámara/posición
    """
    d = {"has_jpg": 0, "has_jpeg": 0, "len_name": 0, "digits_sum": 0}
    if not isinstance(fname, str):
        return d
    name = fname.strip()
    d["has_jpg"] = int(name.lower().endswith(".jpg"))
    d["has_jpeg"] = int(name.lower().endswith(".jpeg"))
    d["len_name"] = len(name)
    digits = re.findall(r"\d+", name)
    d["digits_sum"] = int(np.sum([int(x) for x in digits])) if digits else 0
    return d

# Aplicar preprocesamiento a DataFrame
def apply_preprocessing(df: pd.DataFrame, is_train: bool) -> pd.DataFrame:
    out = df.copy()

    # Asegurar columnas esperadas
    expected = {"ID","filename","growth_stage","damage","season"}
    missing = expected - set(out.columns.str.lower())
    # Normaliza nombres
    out.columns = [c.lower() for c in out.columns]
    # Re-chequeo
    if not {"id","filename","growth_stage","damage","season"}.issubset(set(out.columns)):
        raise ValueError("Faltan columnas clave después de normalizar nombres.")

    # Normalizaciones
    out["season"] = out["season"].map(normalize_season)
    out["stage_code"] = out["growth_stage"].map(normalize_stage)
    out["damage"] = out["damage"].astype(str).str.upper().str.strip()

    # EXTENT: si existe, asegurar rango 0..100
    if "extent" in out.columns:
        out["extent"] = pd.to_numeric(out["extent"], errors="coerce").fillna(0.0).clip(0, 100)

    # Rasgos del filename
    feats = out["filename"].apply(filename_features).apply(pd.Series)
    out = pd.concat([out, feats], axis=1)

    # Mantener solo columnas útiles
    keep_cols = ["id","damage","season","stage_code","has_jpg","has_jpeg","len_name","digits_sum"]
    if "extent" in out.columns:
        keep_cols.append("extent")
    return out[keep_cols]

train_clean = apply_preprocessing(train, is_train=True)
test_clean  = apply_preprocessing(test,  is_train=False)

print(train_clean.head(3))
print(test_clean.head(3))


              id damage  season stage_code  has_jpg  has_jpeg  len_name  \
0  ID_1S8OOWQYCB     WD  SR2020          S        1         0        30   
1  ID_0MD959MIZ0      G  SR2021          V        1         0        26   
2  ID_JRJCI4Q11V      G  LR2020          V        1         0        26   

   digits_sum  extent  
0        7771       0  
1       14726       0  
2        2951       0  
              id damage  season stage_code  has_jpg  has_jpeg  len_name  \
0  ID_ROOWKB90UZ     WD  SR2020          V        1         0        30   
1  ID_PTEDRY0CYM     WD  LR2021          F        1         0        31   
2  ID_5WJXDV96R4     WD  SR2021          V        1         0        30   

   digits_sum  
0        6610  
1       38308  
2       57813  


## Partición con GroupKFold (evitar leakage por ID)


In [ ]:
# Preparar datos para modelado
X_cols_cat = ["damage","season","stage_code"]
X_cols_num = ["has_jpg","has_jpeg","len_name","digits_sum"]
y_col = "extent"

# asseertions
assert y_col in train_clean.columns, "La columna 'extent' debe existir en train."

# limpiar datos
X = train_clean[X_cols_cat + X_cols_num].copy()
y = train_clean[y_col].copy()
groups = train_clean["id"].astype(str).values  

gkf = GroupKFold(n_splits=5)
fold_indices = list(gkf.split(X, y, groups=groups))
print("FOLDS:", len(fold_indices))


FOLDS: 5
